# Is Sympy really 'slow'?
I don't want to be a bad carpenter who blames his tools. Here're my attempts at understanding why my Sympy code is slow. The technical specs of my computer may have a lot to do with the long time things are taking to run, though, any techniques I discover to speed up computation will have a speed up on any system I suppose. 

Ultimately my goal is to have integral and linear algebra routines that can run at sub-second speed, and preferrably a few ms. 

## Potential things to try out 

* ~~I may have been using the wrong Python version of Theano. The [docs](https://theano-pymc.readthedocs.io/en/latest/requirements.html) say official support for Theano is only there for Python <3.6.~~ 
    * ~~install Sympy, Theano nd pyMC3~~ *This doesn't work because Theano didn't seem to recognise 'external' sympy functions, eg. legendre*
    
* With ```lambdify``` use the 'mpmath' module as a backend. This *really* sped up computations, in comparison to the default (sympy?) backend module. 

* Linux specific: change the 'niceness'of the process and so the CPU spends more time on the code.

In [1]:
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects
from gmpy2 import *
import matplotlib.pyplot as plt
import mpmath
from mpmath import mpf
dps = 100; mpmath.mp.dps = dps
import numpy as np
from scipy.special import jv as bessel_firstkind
from symengine import * 
import sympy
from sympy import besselj, symbols, hankel2, legendre, sin, cos, tan, summation, I, oo, diff, pi
from sympy import factor_terms, Matrix
from sympy import Abs, lambdify, integrate, expand,integrate, Integral
from sympy.printing.theanocode import theano_function
from sympy.utilities.autowrap import autowrap
import tqdm
x, alpha, k, m,n,p, r1, R, theta = symbols('x alpha k m n p r1 R theta')
from sympy import N, cse

from sympy.printing.theanocode import theano_function
num_sumterms = 50

In [2]:
hankel2_func = lambdify([n, p], hankel2(n, p), 'mpmath')

In [3]:
r1 = (R*cos(alpha))/cos(theta)
Lm_expr = expand(legendre(m, cos(theta))*(r1**2/R**2)*tan(theta))# 
Lm_wo_leg = (r1**2/R**2)*tan(theta)

In [23]:
subs_dict = {'alpha':mpf(np.radians(60)), 'k':5,'R':mpf(0.1), 'm':20,'n':10, 'theta':mpf(np.radians(5))}

In [5]:
Lm = Integral(Lm_expr, (theta,0,alpha))#.doit(meijerg=True)

In [6]:
Lm_func = lambdify([m, R, alpha], Lm, 'mpmath')

In [7]:
%%time 
Lm_func(subs_dict['m'],subs_dict['R'],subs_dict['alpha'])

CPU times: user 140 ms, sys: 341 µs, total: 140 ms
Wall time: 139 ms


mpf('0.0008098262850926050777410678902535758222607521355914125525164515166435803306093484471288396559860545548289')

In [8]:
# eqn 12.107
Kmn_expr = legendre(n, cos(theta))*legendre(m, cos(theta))*sin(theta)
Kmn = Integral(Kmn_expr, (theta, alpha, pi))#.as_sum(num_sumterms,'midpoint')

In [9]:
kmn_lambdify = lambdify([m,n,alpha],Kmn,'mpmath')

In [10]:
%%time 
kmn_lambdify(subs_dict['m'],subs_dict['n'], subs_dict['alpha'])

CPU times: user 301 ms, sys: 139 µs, total: 301 ms
Wall time: 301 ms


mpf('-0.001727547462648632651566050875068465751494226171518181482997361897449925444398259724115500946819197208954')

In [11]:
Imn_part1 = (n*hankel2(n-1,k*r1)-(n+1)*hankel2(n+1,k*r1))*legendre(n, cos(theta))*cos(theta)
Imn_part2 = n*(n+1)*hankel2(n, k*r1)*(legendre(n-1, cos(theta)-legendre(n+1, cos(theta))))/k*r1
Imn_parts = expand(Imn_part1+Imn_part2)
Imn_expr = expand(Imn_parts*legendre(m,cos(theta))*(r1**2/R**2)*tan(theta))
Imn = Integral(Imn_expr, (theta, 0, alpha)).as_sum(100,'trapezoid')

Imn_lambdify = lambdify([m,n,k,R,alpha], Imn,'mpmath')

In [17]:
from sympy.utilities.autowrap import binary_function, autowrap

In [26]:
%%time
N(Imn_expr.subs(subs_dict),dps)

CPU times: user 21.3 ms, sys: 61 µs, total: 21.4 ms
Wall time: 21.1 ms


0.000000000002229691151349015678457426882362011603184286660331963465160540003610215274991395239675044759754420569 - 1436195310774.684905552399630973877377465816320840487548948951681774490151229010058207461826657247671*I

In [12]:
%%time
u = Imn_lambdify(subs_dict['m'],
             subs_dict['n'],
             subs_dict['k'],
             subs_dict['R'],
            subs_dict['alpha'])
print(u)

(0.0000000000001465641709813984694233211206311576441291691607827907953604710610435374979747704179595561278707871022 - 108911153799.70578339628770975059856066634162555077533516763941362500945881110788626363602684323301j)
CPU times: user 1.37 s, sys: 7.84 ms, total: 1.37 s
Wall time: 1.37 s


In [13]:
M_mn = (Imn + (n*hankel2(n-1,k*R) - (n+1)*hankel2(n+1,k*R) )*Kmn)/(2*n+1)



In [14]:
M_mn_func = lambdify((m,n,k,R,alpha), M_mn,'mpmath')

In [15]:
b = -I*Lm
b_func = lambdify([m,alpha], b,'mpmath')

In [16]:
frequency = 50*10**3 # kHz
vsound = 330 # m/s
wavelength = vsound/frequency
alpha_value = np.radians(60)
k_value = 2*np.pi/(wavelength)
ka = 5
a_value = ka/k_value 
R_value = a_value/mpmath.sin(alpha_value) # m


In [17]:
Nv = 12 + int(2*ka/sin(alpha_value))
M_matrix = mpmath.matrix(Nv,Nv)
b_matrix = mpmath.matrix(Nv,1)


In [18]:
params = {'k':k_value, 'a':a_value, 'R':R_value, 'alpha':alpha_value}

In [19]:
def Mmn(eachm, eachn, params):
    Imn_value = Imn_lambdify(eachm, eachn, params['k'], params['R'], params['alpha'])
    first_hankel = eachn*hankel2_func(eachn-1,params['k']*params['R'])
    second_hankel = (eachn+1)*hankel2_func(eachn+1,params['k']*params['R'])
    Kmn_value = kmn_lambdify(eachm,eachn, params['alpha'])
    return ( Imn_value + (first_hankel - second_hankel)*Kmn_value)/(2*eachn+1)


In [ ]:
%%time
rowcol = [ ]
for eachn in tqdm.trange(Nv):
    for eachm in range(Nv):
        Imn_value = Imn_lambdify(eachm, eachn, params['k'], params['R'], params['alpha'])
        first_hankel = eachn*hankel2_func(eachn-1,params['k']*params['R'])
        second_hankel = (eachn+1)*hankel2_func(eachn+1,params['k']*params['R'])
        Kmn_value = kmn_lambdify(eachm,eachn, params['alpha'])
        M_matrix[eachm, eachm] = ( Imn_value + (first_hankel - second_hankel)*Kmn_value)/(2*eachn+1)


  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
for each_m in range(Nv):
    #M_matrix[each_m, each_n] = M_mn_func(each_m, each_n, k_value, R_value, alpha_value)
    b_matrix[each_m,:] = b_func(each_m, alpha_value)


In [ ]:
a_matrix = mpmath.lu_solve(M_matrix,b_matrix)

In [ ]:
mpmath.residual(M_matrix,a_matrix,b_matrix)

In [ ]:
legendre_func = lambdify((m, x), legendre(m, x),'mpmath')

In [ ]:

def d_theta(angle,k_v,R_v,alpha_v,An):
    num = 4 
    N_v = An.rows
    denom  = (k_v**2)*(R_v**2)*mpmath.sin(alpha_v)**2
    part1 = num/denom
    jn_matrix = np.array([1j**f for f in range(N_v)])
    legendre_matrix = np.array([legendre(n_v, np.cos(angle)) for n_v in range(N_v)])
    part2_matrix = np.column_stack((An, jn_matrix, legendre_matrix))
    part2 = np.sum(np.apply_along_axis(lambda X: X[0]*X[1]*X[2], 1, part2_matrix))
    rel_level = - part1*part2
    return rel_level

def d_zero(k_v,R_v,alpha_v,An):
    num = 4 
    N_v = An.rows
    denom  = (k_v**2)*(R_v**2)*mpmath.sin(alpha_v)**2
    part1 = num/denom
    jn_matrix = np.array([1j**f for f in range(N_v)])
    part2_matrix = np.column_stack((An, jn_matrix))
    part2 = np.sum(np.apply_along_axis(lambda X: X[0]*X[1], 1, part2_matrix))
    rel_level = - part1*part2
    return rel_level

def relative_directionality_db(angle,k_v,R_v,alpha_v,An):
    off_axis = d_theta(angle,k_v,R_v,alpha_v,An)
    on_axis = d_zero(k_v,R_v,alpha_v,An)
    rel_level = 20*mpmath.log10(abs(off_axis/on_axis))
    return rel_level

In [ ]:

angles = np.linspace(0,2*np.pi,200)
dirnlty = [relative_directionality_db(angle_v, k_value, R_value,alpha_value,a_matrix) for angle_v in angles]


In [ ]:
plt.figure()
a0 = plt.subplot(111, projection='polar')
plt.plot(angles, dirnlty)
plt.ylim(-40,0);plt.yticks([0,-10,-20,-30,-40]);
plt.xticks(np.radians(np.arange(0,360,30)));